In [1]:
import sys
import time
import datetime
import json

In [2]:
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

In [3]:
TOPIC_Step3_NAME="jobinja"
TOPIC_Step2_NAME="jobs_info"
KAFKA_SERVER="broker:9092"

In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [5]:
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("job-app") \
    .config("spark.executor.cores","1") \
    .config("spark.cores.max", "1") \
    .config("spark.sql.session.timeZone", "Asia/Tehran") \
    .getOrCreate()  

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-272dfe29-51d1-476a-8769-325ddf5361cb;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 529ms :: artifacts dl 15ms
	:: modules in us

In [6]:
schema = StructType([
    StructField("company", StringType(), True),
    StructField("job_title", StringType(), True),
    StructField("job_category", StringType(), True),
    StructField("job_location", StringType(), True),
    StructField("contract_type", StringType(), True),
    StructField("experience", StringType(), True),
    StructField("salary", StringType(), True),
    StructField("more_details", StringType(), True),
    StructField("passed_day", StringType(), True),
    StructField("job_url", StringType(), True),
])

In [7]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_SERVER) \
  .option("subscribe", TOPIC_Step2_NAME) \
  .option("startingOffsets", "earliest") \
  .load()

In [8]:
df = df.selectExpr("CAST(value AS STRING)")

In [9]:
df = df.select(from_json(col("value"), schema).alias("data")).select("data.*")

In [10]:
df = df.withColumn('timestamp', date_format(current_timestamp(), 'yyyy-MM-dd HH:mm:ss'))

In [ ]:
query = df \
  .selectExpr("CAST(job_url AS STRING) AS key", "to_json(struct(*)) AS value") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_SERVER) \
  .option("topic", TOPIC_Step3_NAME) \
  .option("checkpointLocation", "/opt/spark-notebooks/cheakpoint") \
  .start() \
  .awaitTermination()

23/11/02 12:41:27 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/11/02 12:41:28 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
